Setting up system path and loading config file

In [1]:
isLocalServer = False
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID-model/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print(f'isLocalServer is {isLocalServer}')
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

        # print("System path as follows:")
        # for path in sys.path:
        #     print(f"{path}")

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd

isLocalServer is False
/Volumes/ssd/code/FaceID-model


In [2]:
from data_preprocessing import DatasetBasedOnFolders, DatasetFromNumpy
from data_preprocessing import buildDataLoaders, saveDataloaders
cfg["DatasetBasedOnFolders"]["folder_name_for_source"] = cfg["folder_name_for_source"]
datasetBasedOnFolders = DatasetBasedOnFolders(cfg["DatasetBasedOnFolders"])

device is cpu
image size: 240
Loading faces from ./source/faces_from_photos
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
data length: 399


In [3]:
emb, idx = datasetBasedOnFolders.loadNumpy(return_dataset=True)
path_numpy_emb, path_numpy_label = datasetBasedOnFolders.get_path_to_load_numpy()

dataset = DatasetFromNumpy(path_numpy_emb, path_numpy_label)

(399, 1, 512)
(399,)
(399, 1, 512)
(399,)


In [4]:
dataloaders = buildDataLoaders(dataset, 4, 0.7, 0.2)

dataset length: (399) = tr (279) + val (79) + tt (41)


In [8]:
saveDataloaders(source_folder='data', dataloaders=dataloaders)

saved in ./data/dataloader_train.pt
saved in ./data/dataloader_val.pt
saved in ./data/dataloader_test.pt
